Map each observed healthcare task onto an ONET DWA 
---

- use word2vec to augment the observed task descriptions in order to propse the most similar DWAs (using a string matching function to the Tasks within the DWA). 



By Paul Duckworth 8th Dec 2017.



## OBSERVED TASK DATA

In [1]:
import os
import numpy as np
import pandas as pd
import getpass
import matplotlib.pyplot as plt
import gensim
# import pymed
import time
%matplotlib inline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

datasets = '/home/'+ getpass.getuser() +'/Datasets/'
print datasets

/home/scpd/Datasets/


In [2]:
# d1 = os.path.join(datasets, 'FoHealthcare/FOH Occupations Tasks Features Technology.xlsx')
d1 = os.path.join(datasets, 'FoHealthcare/Tasks_dataset_170718.xlsx')

excel_doc = pd.ExcelFile(d1)
# dataset = excel_doc.parse("Title, Tasks, Features").fillna("-")
dataset = excel_doc.parse("Tasks").fillna("-")
dataset.rename(columns = {'Occupation title':'Observed Occupation'}, inplace = True)
dataset['Task ID'] = dataset.index
data = dataset[['Observed Occupation', 'Task', 'Task ID']]
data.head()

,Observed Occupation,Task,Task ID
0,Administrator,Medical Coding of letters and other documents,0
1,Administrator,Register new Patients,1
2,Administrator,Use software to convert printed letters into t...,2
3,Administrator,Refer to child immunization targets in open ex...,3
4,Administrator,"Write letters for secondary care, other GPs, o...",4


In [3]:
# # Test dataset with added Task "context".
# d2 = os.path.join(datasets, 'FoHealthcare/expanded tasks descriptions_for matching DWAs.csv')
# data = pd.read_csv(d2)
# data.rename(columns = {'Occupation title':'Observed Occupation', 'Task keywords/context':'Context'}, inplace = True)
# data['Task ID'] = data.index
# data.head()

## Observed Vocabulary: 

In [4]:
cv1 = CountVectorizer(stop_words='english') #max_df=0.95, min_df=2, max_features=n_features))

# Each Task is represented by a vector of Words over vocabulary
observed_tf = cv1.fit_transform(data['Task'].values)
observed_vocab = cv1.get_feature_names()

print data['Task'][0]
print observed_tf[0]
print len(observed_vocab), observed_vocab[:100]

Medical Coding of letters and other documents
  (0, 99)	1
  (0, 173)	1
  (0, 55)	1
  (0, 185)	1
353 [u'account', u'accounting', u'additional', u'address', u'addressing', u'administer', u'administrative', u'admit', u'advice', u'agenda', u'ambulance', u'answer', u'appointment', u'appointments', u'approve', u'arise', u'assesment', u'assign', u'attend', u'audio', u'audit', u'audits', u'authority', u'authorizations', u'b12', u'banages', u'bills', u'blood', u'bloodpressure', u'book', u'bookable', u'building', u'business', u'called', u'canal', u'care', u'case', u'cases', u'cash', u'certain', u'changes', u'check', u'checking', u'checklist', u'checks', u'checkups', u'child', u'chronic', u'clean', u'cleaning', u'clinic', u'clinical', u'clinics', u'coded', u'coder', u'coding', u'colleagues', u'commissioning', u'communicate', u'communication', u'complaints', u'computer', u'condition', u'conduct', u'connecting', u'consult', u'consultation', u'contact', u'control', u'convert', u'cost', u'counseling'

## ONET TASK DATA

In [5]:
# #Task DWAs (detailed work activitiy code):
taskDWA = pd.read_table(os.path.join(datasets, 'ONET/databases/db2016/Tasks to DWAs.txt'), sep='\t')
DWArefs = pd.read_table(os.path.join(datasets, 'ONET/databases/db2016/DWA Reference.txt'), sep='\t')
DWA_sup = taskDWA[['Task ID', 'DWA ID']].merge(DWArefs[['DWA ID', 'IWA ID', 'DWA Title']], on=['DWA ID'])

# onet_tasks_dwa = onet_tasks[['Task ID', 'Task']].merge(DWA_sup, on=['Task ID'])\
#                                                          .sort_values(by='Task ID')\
#                                                          .reset_index().drop('index', axis=1)
# print "Merged Tasks = ", onet_tasks_dwa.shape

# Just ONET Tasks
onet_tasks = pd.read_table(os.path.join(datasets, 'ONET/databases/db2016/Task Statements.txt'), sep='\t')
print "all Tasks = ", onet_tasks.shape
onet_tasks = onet_tasks[['Task ID', 'Task']]

onet_tasks=onet_tasks.sort_values(by='Task ID').reset_index().drop('index', axis=1)
onet_tasks.head()

all Tasks =  (19566, 7)


,Task ID,Task
0,1,Resolve customer complaints regarding sales an...
1,2,Monitor customer preferences to determine focu...
2,3,Direct and coordinate activities involving sal...
3,4,Determine price schedules and discount rates.
4,5,Review operational records and reports to proj...


## ONET Vocabulary: 

In [6]:
cv2 = CountVectorizer(stop_words='english') #max_df=0.95, min_df=2, max_features=n_features)#, )

# Each Task is represented by a vector of Words over vocabulary
onet_tf = cv2.fit_transform(onet_tasks['Task'].values)
onet_vocab = cv2.get_feature_names()

print onet_tasks['Task'][0]
print onet_tf[0]
print len(onet_vocab), onet_vocab[:100]

Resolve customer complaints regarding sales and service.
  (0, 9682)	1
  (0, 9399)	1
  (0, 8823)	1
  (0, 2125)	1
  (0, 2710)	1
  (0, 9083)	1
12085 [u'10', u'24', u'3d', u'4d', u'abandoned', u'abatement', u'abatements', u'abbreviations', u'abdominal', u'abilities', u'ability', u'ablation', u'able', u'abnormal', u'abnormalities', u'aboard', u'aboveground', u'abraders', u'abrading', u'abrasion', u'abrasions', u'abrasive', u'abrasives', u'abreast', u'abroad', u'abscesses', u'absence', u'absences', u'absenteeism', u'absorbers', u'absorbing', u'absorption', u'abstract', u'abstracting', u'abstracts', u'abundance', u'abuse', u'abused', u'academia', u'academic', u'academy', u'accelerant', u'accelerated', u'accelerator', u'accelerators', u'accenting', u'accept', u'acceptability', u'acceptable', u'acceptance', u'accepted', u'accepting', u'access', u'accessed', u'accesses', u'accessibility', u'accessible', u'accessing', u'accessories', u'accessory', u'accident', u'accidental', u'accidents', u'acco

## Model

- https://code.google.com/archive/p/word2vec/
- 3 million unique words and phrases that they trained on roughly 100 billion words from a Google News dataset.



In [7]:
location = '/home/'+ getpass.getuser() +'/Software/GoogleNews-vectors-negative300.bin'
model = gensim.models.KeyedVectors.load_word2vec_format(location, binary=True)

## Similarity between pairwise words in 2 vocabularies:

In [8]:
word_sims = np.zeros([len(observed_vocab), len(onet_vocab)])
for cnt, word in enumerate(observed_vocab):
    print cnt, word,
    ss = []
    for j in onet_vocab:
        s = 0.0
        try:
            s = model.similarity(word, j)
            if s < 0:
                s = 0.0
        except:
            pass
        ss.append(s)       
    word_sims[cnt] = ss    

0 account 1 accounting 2 additional 3 address 4 addressing 5 administer 6 administrative 7 admit 8 advice 9 agenda 10 ambulance 11 answer 12 appointment 13 appointments 14 approve 15 arise 16 assesment 17 assign 18 attend 19 audio 20 audit 21 audits 22 authority 23 authorizations 24 b12 25 banages 26 bills 27 blood 28 bloodpressure 29 book 30 bookable 31 building 32 business 33 called 34 canal 35 care 36 case 37 cases 38 cash 39 certain 40 changes 41 check 42 checking 43 checklist 44 checks 45 checkups 46 child 47 chronic 48 clean 49 cleaning 50 clinic 51 clinical 52 clinics 53 coded 54 coder 55 coding 56 colleagues 57 commissioning 58 communicate 59 communication 60 complaints 61 computer 62 condition 63 conduct 64 connecting 65 consult 66 consultation 67 contact 68 control 69 convert 70 cost 71 counseling 72 cqc 73 cqrs 74 create 75 data 76 database 77 dbs 78 deal 79 declaration 80 deductions 81 deprescribing 82 desk 83 desktop 84 diabeies 85 diabetic 86 diabetics 87 diagnose 88 diar

## Average Similarity between observed tasks and ONET tasks: 

In [9]:
print onet_tasks['Task'].values[0]
print np.mean(word_sims[0][onet_tf[0].indices])
print  data['Task'][71]

Resolve customer complaints regarding sales and service.
0.11940108994614163
Manage pension schemes


In [10]:
word_sims[[ 40, 111,  12,  20,  35,  33,  42, 126 , 16, 113]]  #.T[[0,10]]
print observed_tf[0].indices
print onet_tf[0].indices

[ 99 173  55 185]
[9682 9399 8823 2125 2710 9083]


In [11]:
print np.mean( word_sims[[ 5, 17]].T[[9682, 9399, 8823, 2125, 2710, 9083]])
sum(sum(word_sims[[ 5, 17]].T[[9682, 9399, 8823, 2125, 2710, 9083]])) / 12.

0.08799937326066447


0.08799937326066444

In [12]:
    
# onet_tasks_dwa = onet_tasks[['Task ID', 'Sims']].merge(DWA_sup, on=['Task ID'])\
#                                                         .sort_values(by='Task ID')\
#                                                         .reset_index().drop('index', axis=1)
        
# x = onet_tasks_dwa.groupby(['DWA ID', 'DWA Title']).mean().reset_index()[['DWA ID', 'DWA Title', 'Sims']].sort_values(by='Sims',ascending=False)[:10]
# x


In [16]:
## Output Format:
n_keep_dwas = 200
columns = ['Occupation', 'Task ID', 'Task', 'DWA Title', 'Please Select', 'Relevance \nScore', 'DWA ID'] 

## Big Loop over the observed Tasks
# search_for_tasks = [66, 125, 99, 134, 114, 89, 7, 24]
search_for_tasks = [84]

# search_for_tasks = [i-4 for i in search_for_tasks] # the row numbers in Excel are off by 2 to the task count. 
print search_for_tasks

xls_path = os.path.join(datasets, 'FoHealthcare/recommended_DWA_matches_subset.xlsx')
writer = pd.ExcelWriter(xls_path, engine='xlsxwriter')

for task_cnt, obs_task in enumerate(observed_tf):  
     
    if task_cnt not in search_for_tasks: continue
    print task_cnt,
    
    ## Get rows corresponding to the observed task
    row_inds = obs_task.indices
    
    onet_similarities = []
    for onet_task in onet_tf:
        
        ## Get the columns corresponding to the words in the ONET tasks
        cols = onet_task.indices
        onet_similarities.append(np.mean(word_sims[row_inds].T[cols]))
        
    ## Merge (Overwrite) the similarity of the observed task onto the ONET dataframe
    onet_tasks['Sims'] = onet_similarities

    ## Merge DWA attributes on
    onet_tasks_dwa = onet_tasks[['Task ID', 'Sims']].merge(DWA_sup, on=['Task ID'])\
                                                            .sort_values(by='Task ID')\
                                                            .reset_index().drop('index', axis=1)

    ## Average Similarity over DWAs
    x = onet_tasks_dwa.groupby(['DWA ID', 'DWA Title']).mean()\
                                                    .reset_index()\
                                                    [['DWA ID', 'DWA Title', 'Sims']]\
                                                    .sort_values(by='Sims',ascending=False)[:n_keep_dwas]

    observed_task = data['Task'].values[task_cnt]
    observed_occu = data['Observed Occupation'].values[task_cnt]
    observed_id = data['Task ID'].values[task_cnt]
    
    print (observed_occu, observed_id, observed_task)
    ## Create the first output row: 
    ms = [(observed_occu, observed_id, observed_task, 
           x['DWA Title'].values[0], "-", x['Sims'].values[0], x['DWA ID'].values[0] )] 

    ## Create the subsequent output rows: 
    for cnt, (i, dwa_id, dwa, s) in enumerate(x.itertuples()):
        if cnt == 0: continue

        ms.extend([('-', '-', '-',
            x['DWA Title'].values[cnt], "-",  x['Sims'].values[cnt], 
            x['DWA ID'].values[cnt] )])

    ## Create a tab in the excel document 
    tab_name = str(task_cnt+2)
    
    df_ = pd.DataFrame(data = np.array(ms), columns=columns)
    df_.to_excel(writer, '%s' % tab_name)
    
    ## Format the Excel Sheet: 
    workbook  = writer.book
    format = workbook.add_format()
    format.set_text_wrap() # wraps text

    worksheet = writer.sheets['%s' % tab_name]
    worksheet.set_row(0, 30)  # set the height of the first row
    worksheet.set_row(1, 70)  # set the height of the first row
    
    worksheet.set_column('A:A', 5, format)  #formats a column and specifies width
    worksheet.set_column('B:B', 20, format)
    worksheet.set_column('C:C', 10, format)
    worksheet.set_column('D:D', 45, format)
    worksheet.set_column('E:E', 60, format)
    worksheet.set_column('F:F', 10, None)
    worksheet.set_column('G:G', 10, None)
    worksheet.set_column('H:H', 10, None)
    
writer.save()
print "finished"

[84]
84 (u'Practice manager', 84, u'Having practice staff take online training ')
finished


In [11]:
# list1, list2 = zip(*sorted(zip(matched_tasks[t], onet_tasks['Task'].values), reverse=True))